# Estimating the dispersion of image matching displacements
Functions and algorithms within the **dhdt** framework are for a large part dependent upon pattern matching. When such algorithms are formulated as a convolution, the cross-correlation function can have the form of a peak. The width of the peak can then be used as a proxy for precision.

Such a describtor of precision is of interest when information is extracted from such a velocity field. Since errors will propagate in any derived product. In the following, an example is given from a dataset of two images of Sentinel-2 over a large outlet glacier in Greenland. 

This work is a demonstration of the basic functionality, and a more detailed describtion can be found in [Altena et al. 2021](https://doi.org/10.5194/tc-2021-202).

### import of libraries
First we need to import some generic libraries

In [5]:
import os
import urllib
import numpy as np
import matplotlib.pyplot as plt

import specific libraries

In [8]:
from dhdt.generic.mapping_io import read_geo_image, read_geo_info 
from dhdt.generic.mapping_io import read_geo_image, read_geo_info 

ModuleNotFoundError: No module named 'dhdt'

### data preparation
The satellite data is situated in the cloud (pun intended), but these can be downloaded to a local drive. Hence the following code sniplet looks if such data is already available, if not it is downloaded.

In [6]:
dat_dir = '/Users/Alten005/dump-dir'
dat_url = 'https://surfdrive.surf.nl/files/index.php/s/7filDMbYkxiavk3'

I1_name = 'T22WEB_20200730T151921_B04_clip.jp2'
I2_name = 'T22WEB_20200720T151911_B04_clip.jp2'

# do downloading if files are not present
if not os.path.exists(os.path.join(dat_dir, I1_name)):
    file_grab = urllib.URLopener()
    print('busy downloading files') # dowload Sentinel-2 L1C imagery data
    file_grab.retrieve(os.path.join(dat_url, I1_name),
                       os.path.join(dat_dir, I1_))
    file_grab.retrieve(os.path.join(dat_url, I2_name),
                       os.path.join(dat_dir, I2_name))
    print('files dowloaded')
else:
    print('files are already present')

NameError: name 'rgi_name' is not defined

### main processing
The imagery will be matched, here certain parameters should be given.

In [9]:
temp_radius,sear_radius = 10, 40

Now it is possible to read the imagery

In [ ]:
I1 = read_geo_image(os.path.join(dat_dir, I1_name))[0]
I2 = read_geo_image(os.path.join(dat_dir, I2_name))[0]

### post-processing
The advantage of a formulation for dispersion can be found in the estimation of product quality through error propagation. Hence in the this later part, the estimated co-variances will be used to help reflect upon the data quality of calculated strain rates.

Strain rates are of interest to glaciologists, as ice can behave both as a viscous fluid, as well as, a brittle material. Typically, when a specific extensive force is applied, a crevasse will emerge. 